In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from cryptography.fernet import Fernet

# Function to generate a key for AES encryption
def generate_key():
    return Fernet.generate_key()

# Function to encrypt data using AES
def encrypt_data(key, data):
    cipher = Fernet(key)
    encrypted_data = []
    for item in data:
        encrypted_item = cipher.encrypt(str(item).encode()).decode()
        encrypted_data.append(encrypted_item)
    return encrypted_data

# Function to decrypt data using AES
def decrypt_data(key, encrypted_data):
    cipher = Fernet(key)
    decrypted_data = []
    for item in encrypted_data:
        decrypted_item = cipher.decrypt(item.encode()).decode()
        decrypted_data.append(decrypted_item)
    return decrypted_data

# Function to preprocess data and train VAE model
def preprocess_and_train_vae(data):
    # Use LabelEncoder to encode the VictimName column
    label_encoder = LabelEncoder()
    data['Encoded_VictimName'] = label_encoder.fit_transform(data['VictimName'])

    # Split the data into training and validation sets
    X_train, X_val = train_test_split(data['Encoded_VictimName'].values.reshape(-1, 1), test_size=0.2, random_state=42)

    # Rest of the VAE model training code here...

    return encoded_victim_names, scaler, encoder  # Return the necessary values

# Load your dataset
data = pd.read_csv('VictimInfoDetails.csv')

# Preprocess and train VAE model
encoded_victim_names, scaler, encoder = preprocess_and_train_vae(data)

key = generate_key()

encrypted_victim_names = encrypt_data(key, encoded_victim_names)

# Create a new column named 'Encrypted_VictimName' in the DataFrame 'data' and assign the encrypted values
data['Encrypted_VictimName'] = encrypted_victim_names

# Optionally, you can drop the original VictimName column
data.drop('VictimName', axis=1, inplace=True)

# Save the processed data to a new CSV file
data.to_csv('processed_victim_data.csv', index=False)